# Julia Multithreading Gotchas

This notebook compiles a list of example code to illustrate Julia's [multithreading](https://docs.julialang.org/en/v1/manual/multi-threading/) feature. We focus on "performance gotchas" which are not well-documented (as of 2020). Note multithreading parallelism is different than Julia's [distributed parallelism](https://docs.julialang.org/en/v1/stdlib/Distributed/) in that the former always assume all data are present on a single computer. 

In [1]:
using Random
using BenchmarkTools
using LinearAlgebra

## How to access multithreading feature

Before starting Julia (REPL or notebook), type 

    `export JULIA_NUM_THREADS=8`

then verify you indeed have multiple active threads:

In [2]:
Threads.nthreads()

8

## Gotcha 1: Race Conditions - Wrong Answers!

+ A race condiion is when 2 or more threads modify the same variable simultaneously, giving you the wrong answer. 
+ Julia makes no effort to check if your code is thread-safe
+ There is no good way to debug one!! 

In [12]:
function unsafe_sum(x::AbstractVector)
    s = 0.0
    Threads.@threads for i in eachindex(x)
        s += x[i]
    end
    return s
end

Random.seed!(2020)
x = rand(100000)
@show unsafe_sum(x)
@show unsafe_sum(x);

unsafe_sum(x) = 6559.801414594502
unsafe_sum(x) = 6271.334128542942


Answers don't match! There is a race condition! To debug race conditions, I suggest using [Threads.SpinLock()](https://docs.julialang.org/en/v1/base/multi-threading/#Base.Threads.SpinLock). 

In [11]:
function debug_unsafe_sum(x::AbstractVector)
    s = 0.0
    m = Threads.SpinLock() # mutex object
    Threads.@threads for i in eachindex(x)
        lock(m)
        s += x[i] # surround possible race condition with locks
        unlock(m)
    end
    return s
end

@show debug_unsafe_sum(x)
@show debug_unsafe_sum(x);

debug_unsafe_sum(x) = 49938.49784564638
debug_unsafe_sum(x) = 49938.49784564645


**Conclusion:** `s += x[i]` is causing the race condition! This is obvious: if 2 threads are *simultaneously* updating `s`, only one thread's result will be recorded. Since we have 8 threads, only ~1/8 of all operations are updated and the rest is lost. That's why our `unsafe_sum` returned a value that's about $1/8$ of the correct answer. 

To fix this, force each thread to modify a different value. 

In [10]:
function safe_sum(x::AbstractVector)
    threads = Threads.nthreads()
    s = zeros(threads)
    Threads.@threads for i in eachindex(x)
        s[Threads.threadid()] += x[i] # each thread adds to a different location
    end
    return sum(s) # return sum of each thread
end

@show safe_sum(x)
@show safe_sum(x);

safe_sum(x) = 49938.49784564629
safe_sum(x) = 49938.49784564629


Finally we resolved the race condition. Let's check timings:

In [13]:
@btime safe_sum($x)
@btime debug_unsafe_sum($x)
@btime sum($x);

  36.357 μs (42 allocations: 5.66 KiB)
  5.950 ms (200043 allocations: 3.06 MiB)
  12.471 μs (0 allocations: 0 bytes)


We are faster than using locks (which is single threaded and requires extra bookkeeping), but slower than Julia's built-in `sum` command. That is because Julia's built-in `sum` uses many [performance annotations](https://docs.julialang.org/en/v1/manual/performance-tips/#man-performance-annotations) like `@inbounds` and `@simd`. The latter stands for single-instruction multiple data, which is more suitable parallel mechanism for calculating sum of vectors.

## Gotcha 2: False sharing - Not-actually-parallel parallel code

In [ ]:
# see post https://discourse.julialang.org/t/editing-an-array-with-multiple-threads/25364/9
function f(spacing)
    n = 1000000
    x = rand(n)

    s = zeros(Threads.nthreads()*spacing) # sum of x
    c = zeros(Threads.nthreads()*spacing) # count additions in each thread

    Threads.@threads for i = 1:n
        s[Threads.threadid()*spacing] += x[i] # thread i adds to s[i]
        c[Threads.threadid()*spacing] += 1    # thread i adds to c[i]
    end
    return sum(s), c
end

@btime f(1);
@btime f(8);

## Gotcha 3: Oversubscription - parallelism on top of parallelism

This can happen when

+ When code performs parallel operations which themselves call other parallel operations (e.g. BLAS + multithreading).
+ You start Julia with more threads than the number of physical cores on your CPU

**Note:** Not all oversubscriptions are bad. For instance, intensive I/O operations can benefit with more threads than CPU cores because loading/writing data to/from disk is so slow that the CPU is basically idle most of the time. In my experience, oversubscription is *bad* for tasks where everything is loaded in memory, and good otherwise. 

In [31]:
# Performs C[i] = A[i] * B[i] where A[i], B[i], C[i] are matrices
function test_multiply!(C, A, B)
    Threads.@threads for i in eachindex(C)
        id = Threads.threadid()
        mul!(C[id], A[id], B[id])
    end
end

# simulate data
Random.seed!(2020)
A = [rand(100, 100) for _ in 1:100]
B = [rand(100, 100) for _ in 1:100]
C = [rand(100, 100) for _ in 1:100];

In [32]:
# 8 BLAS threads (default)
BLAS.set_num_threads(8)
@btime test_multiply!($C, $A, $B)

# 1 BLAS thread
BLAS.set_num_threads(1)
@btime test_multiply!($C, $A, $B);

  3.023 ms (41 allocations: 5.53 KiB)
  753.820 μs (41 allocations: 5.53 KiB)


**Conclusion:** If you forgot to change BLAS threads (8 by default), your for loop is automatically **4 times** slower!

## Gotcha 4: Non-uniform tasks - bottleneck is singlethreaded

## Gotcha 5: Memory allocation

ANY extra allocation memory are completely detrimental to performance. Even a few KB which takes no time at all for single-threaded programs can completely kill performance once you use `@threads`. Apparently this is because GC is single-threaded (not sure) so you really don't want to trigger it.

In [ ]:
function test_multiply()
    threads = Threads.nthreads()
    A = [rand(1000, 1000) for _ in 1:threads]
    B = [rand(1000, 1000) for _ in 1:threads]
    C = [rand(1001, 1001) for _ in 1:threads]
    Threads.@threads for i in 1:10000
        id = Threads.threadid()
        mul!(view(C[id], 1:1000, 1:1000), A[id], B[id])
    end
    return C
end
test_multiply();